In [17]:
# ===============================
# CELL 0: GLOBAL SETUP (HIGH ACC)
# ===============================

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

torch.backends.cudnn.benchmark = True

DATA_PATH = "/kaggle/input/cipher"

BATCH_SIZE = 128
NUM_WORKERS = 4

EPOCHS_L1 = 15
EPOCHS_L2 = 20
EPOCHS_L3 = 15


Using device: cuda


In [18]:
# ===============================
# CELL 1: DATA LOADING
# ===============================

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.AutoAugment(transforms.AutoAugmentPolicy.CIFAR10),
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),
                         (0.2470,0.2435,0.2616))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),
                         (0.2470,0.2435,0.2616))
])

train_dataset = CIFAR10(DATA_PATH, train=True, transform=train_transform, download=False)
test_dataset  = CIFAR10(DATA_PATH, train=False, transform=test_transform, download=False)

train_loader = DataLoader(
    train_dataset, BATCH_SIZE, shuffle=True,
    num_workers=NUM_WORKERS, pin_memory=True
)

test_loader = DataLoader(
    test_dataset, BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=True
)

print("Train:", len(train_dataset), "Test:", len(test_dataset))


Train: 50000 Test: 10000


In [19]:
# ===============================
# CELL 2: LEVEL 1 - BASELINE
# ===============================

model = models.resnet50(weights="IMAGENET1K_V1")
model.fc = nn.Linear(model.fc.in_features, 10)
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS_L1)

for epoch in range(EPOCHS_L1):
    model.train()
    correct = 0

    for x, y in train_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        loss = criterion(model(x), y)
        loss.backward()
        optimizer.step()
        correct += (model(x).argmax(1) == y).sum().item()

    scheduler.step()
    print(f"Epoch {epoch+1} Acc:", 100*correct/len(train_dataset))

# Test
model.eval()
correct = 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        correct += (model(x).argmax(1) == y).sum().item()

level1_acc = 100 * correct / len(test_dataset)
print("LEVEL 1 ACC:", level1_acc)


Epoch 1 Acc: 70.064
Epoch 2 Acc: 77.156
Epoch 3 Acc: 79.87
Epoch 4 Acc: 81.646
Epoch 5 Acc: 82.882
Epoch 6 Acc: 83.64
Epoch 7 Acc: 84.3
Epoch 8 Acc: 84.618
Epoch 9 Acc: 85.476
Epoch 10 Acc: 86.11
Epoch 11 Acc: 86.322
Epoch 12 Acc: 86.454
Epoch 13 Acc: 86.796
Epoch 14 Acc: 86.734
Epoch 15 Acc: 86.832
LEVEL 1 ACC: 89.48


In [20]:
# ===============================
# CELL 3: LEVEL 2 - IMPROVED
# ===============================

model2 = models.resnet18(weights="IMAGENET1K_V1")
model2.fc = nn.Linear(model2.fc.in_features, 10)
model2 = model2.to(DEVICE)

optimizer = optim.AdamW(model2.parameters(), lr=3e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS_L2)

for epoch in range(EPOCHS_L2):
    model2.train()
    for x, y in train_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        loss = criterion(model2(x), y)
        loss.backward()
        optimizer.step()
    scheduler.step()

# Test
model2.eval()
correct = 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        correct += (model2(x).argmax(1) == y).sum().item()

level2_acc = 100 * correct / len(test_dataset)
print("LEVEL 2 ACC:", level2_acc)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 188MB/s] 


LEVEL 2 ACC: 87.26


In [21]:
# ===============================
# CELL 4: LEVEL 3 - EFFICIENTNET
# ===============================

model3 = models.efficientnet_b0(weights="IMAGENET1K_V1")
model3.classifier[1] = nn.Linear(model3.classifier[1].in_features, 10)
model3 = model3.to(DEVICE)

optimizer = optim.AdamW(model3.parameters(), lr=3e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS_L3)

for epoch in range(EPOCHS_L3):
    model3.train()
    for x, y in train_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        loss = criterion(model3(x), y)
        loss.backward()
        optimizer.step()
    scheduler.step()

# Test
model3.eval()
correct = 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        correct += (model3(x).argmax(1) == y).sum().item()

level3_acc = 100 * correct / len(test_dataset)
print("LEVEL 3 ACC:", level3_acc)


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 145MB/s]


LEVEL 3 ACC: 84.02


In [22]:
# ===============================
# CELL 5: LEVEL 4 - ENSEMBLE
# ===============================

models_list = [model, model2, model3]
for m in models_list:
    m.eval()

correct = 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        preds = torch.stack([m(x).softmax(1) for m in models_list]).mean(0)
        correct += (preds.argmax(1) == y).sum().item()

ensemble_acc = 100 * correct / len(test_dataset)
print(" ENSEMBLE ACCURACY:", ensemble_acc)


 ENSEMBLE ACCURACY: 90.1
